In [1]:
import tensorflow as tf
print('tf version:', tf.__version__)
from lagrange_lite.utils import io
from lagrange_lite.tensorflow import train
#from lagrange_lite.tensorflow import aop



tf version: 1.15.0


ModuleNotFoundError: No module named 'collector'

In [5]:
tf.app.flags.DEFINE_string('train_paths', 'hdfs://haruna/recommend/data/group_comment/reckon_offline/tfrecord/{20191009}/part-*', 'HDFS paths to input files.')
tf.app.flags.DEFINE_string("eval_paths", 'hdfs://haruna/recommend/data/group_comment/reckon_offline/tfrecord/{20190902}/part-*', "eval data path")
tf.app.flags.DEFINE_string('model_path', 'hdfs://haruna/recommend/data/group_comment/reckon_offline/tfrecord_model/1/', 'Where to write output files.')
tf.app.flags.DEFINE_string('last_model_path', '', 'Model path for the previous run.')
tf.app.flags.DEFINE_integer("train_epochs", 4, "train epochs")
tf.app.flags.DEFINE_integer("batch_size", 512, "batch size")
tf.app.flags.DEFINE_float("learning_rate", 1e-4, "train learning rate")
tf.app.flags.DEFINE_float("dropout", 0.8, "dropout")
tf.app.flags.DEFINE_float("clip_norm", 10.0, "clip norm")
tf.app.flags.DEFINE_integer("num_cols", 247, "num cols")


tf.app.flags.DEFINE_string('f', '', 'kernel') ## add for jupyter, or error

FLAGS = tf.app.flags.FLAGS

print(FLAGS.model_path)
print(FLAGS.train_paths)
tf.io.gfile.makedirs(FLAGS.model_path)


/Users/aodandan/data/dnn/model/
/Users/aodandan/data/dnn/train/{20191008}/part-*


In [ ]:
def build_feature_columns(num_cols=247):
    columns = []
    for i in range(num_cols):
        num_column = tf.feature_column.numeric_column("slot_%s"%i)
        columns.append(num_column)
    return columns
build_feature_columns(10)

In [ ]:
def read_data(paths, batch_size=512, num_epochs=1, shuffle=False, buffer_size=50000, num_cols=247, num_parallels=1, num_workers=1, worker_index=0):
    def parse(value):
        desc = {
                'slot_%s'%i: tf.FixedLenFeature([1], tf.float32, default_value=0.0) for i in range(0, num_cols)
            }
        desc["label"] = tf.FixedLenFeature([1], tf.int64, default_value=0)
        example = tf.parse_single_example(value, desc)
        label = example["label"]
        label = tf.cast(label,tf.int32)
        del example["label"]
        return example, label
    
    print('paths:', paths)
    expanded_path = io.expand_paths(paths)
    print('expanded_path:', expanded_path)
    data_files = tf.data.Dataset.list_files(expanded_path)
    print('data_files:', data_files)

    dataset = tf.data.TFRecordDataset(data_files, num_parallel_reads=num_parallels)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=buffer_size)
    # dataset = dataset.shard(num_workers, worker_index)

    return dataset.map(parse, num_parallel_calls=num_parallels).repeat(num_epochs).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

train_data = read_data(FLAGS.train_paths, batch_size=5, num_cols=2)
with tf.Session() as sess:
    batch_item = train_data.take(1)
    it = tf.compat.v1.data.make_one_shot_iterator(batch_item)
    el = it.get_next()
    res = sess.run(el)
    print(res)


In [ ]:
def build_model(FLAGS):
    print(FLAGS.model_path)
    print(FLAGS.last_model_path)
    print(FLAGS.learning_rate)
    print(FLAGS.clip_norm)
    print(FLAGS.num_cols)
    print(FLAGS.dropout)
    checkpoint_dir = FLAGS.model_path
    if FLAGS.last_model_path and not tf.train.latest_checkpoint(checkpoint_dir):
        warmup_dir = FLAGS.last_model_path
    else:
        warmup_dir = None

    my_optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)
    my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, FLAGS.clip_norm)

    return tf.estimator.DNNClassifier(
        feature_columns=build_feature_columns(FLAGS.num_cols),
        hidden_units=[256, 64],
        optimizer = my_optimizer,
        n_classes = 2,
        dropout=FLAGS.dropout,
        config=tf.estimator.RunConfig(model_dir=checkpoint_dir),
        warm_start_from=warmup_dir)

model = build_model(FLAGS)


In [ ]:
def serving_input_receiver_fn(num_cols=247):
    features = {}
    for i in range(num_cols):
        fname = "slot_%s"%(i)
        features[fname] = tf.placeholder(tf.float32, shape=[None], name=fname)
    return tf.estimator.export.ServingInputReceiver(features, features)

receiver_fn = serving_input_receiver_fn(3)
print(type(receiver_fn))
print(receiver_fn.features)

In [ ]:
def train_input_fn():
    return read_data(FLAGS.train_paths,
                     batch_size=FLAGS.batch_size,
                     num_epochs=FLAGS.train_epochs,
                     shuffle=True,
                     num_cols=FLAGS.num_cols,
                     num_parallels=1)

def eval_input_fn():
    return read_data(FLAGS.eval_paths,
                     batch_size=FLAGS.batch_size,
                     num_cols=FLAGS.num_cols,
                     num_parallels=1)




In [2]:
def main(_):
    model = build_model(FLAGS)
    
    auc_hook = train.BinaryMetricHook(
        num_instances=10000, # 每积攒了不少于 1000 个样本便触发计算
        log_auc_steps=50,
        score_tensor_name='dnn/head/predictions/logistic:0', # 预估分 Tensor 的名称，形状应为 [?] 或 [?,1]
        label_tensor_name='IteratorGetNext:247' # 标签 Tensor 的名称，形状应为 [?] 或 [?,1]
    )
    hooks = [auc_hook]
    train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=6000000, hooks=hooks)
    
    feature_spec = tf.feature_column.make_parse_example_spec(build_feature_columns())
    export_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)
    exporter = tf.estimator.FinalExporter('gandalf', export_input_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn, steps=100, throttle_secs=10, exporters=[exporter])
    
    tf.estimator.train_and_evaluate(model, train_spec, eval_spec)
    
    model.export_saved_model(FLAGS.model_path + '/saved_model', 
                serving_input_receiver_fn=serving_input_receiver_fn)

tf.app.run()


NameError: name 'build_model' is not defined